
## Upload YouTube Videos to the Reading Material Folder Manually


In this notebook, we will take URLs of videos, convert them to mp3s, upload them to Google Drive, trash them, and delete them.


We will start by ensuring that all the necessary python libraries are installed.

In [40]:

!pip install --upgrade --quiet youtube_dl

In [42]:

!pip install --upgrade --quiet PyDrive


Next, we will prove to ourselves that we have ffmpeg correctly installed.

In [35]:

# https://github.com/adaptlearning/adapt_authoring/wiki/Installing-FFmpeg
!ffmpeg -version

ffmpeg version N-90173-gfa0c9d69d3 Copyright (c) 2000-2018 the FFmpeg developers
built with gcc 7.3.0 (GCC)
configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-bzlib --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libmfx --enable-amf --enable-cuda --enable-cuvid --enable-d3d11va --enable-nvenc --enable-dxva2 --enable-avisynth
libavutil      56.  7.101 / 56.  7.101
libavcodec     58. 13.100 / 58. 13.100
libavformat    58. 10.100


Performing authentication with Google Drive requires we keep our client id and consumer secret in a json file. Keep your secrets off the internet!

In [6]:

!start %windir%\explorer.exe "C:\Users\dev\Documents\repositories\notebooks\Miscellaneous\json"


We also make sure line 286 of pydrive's files.py file reads <code>self.content.close()</code> before we run the trash cell.

In [43]:

!"C:\Program Files\Notepad++\notepad++.exe" "C:\Users\dev\Anaconda3\Lib\site-packages\pydrive\files.py"


Before we start the download process, let's go over setting a few variables:

* **The downloads folder:** This is where all the messy work will be done. All mp3 files and any intermediate files will be deleted out of this folder by the end.
* **The URL list:** This is where we place our YouTube video URLs. Make sure you remove any <code>time_continue=</code> parameters in the URL or the mp3 will get cut off.
* **The target folder ID:** This is the folder in Google Drive where our uploads will end up.

In [3]:

import youtube_dl

downloads_folder = r'Downloads\\'
youtube_url_list = ['https://www.youtube.com/watch?v=_nHCM1D9uwk', 'https://www.youtube.com/watch?v=PkNv4LFpGf4',
                    'https://www.youtube.com/watch?v=B2EXSVePg1A', 'https://www.youtube.com/watch?v=s6LlkCsQSq0',
                    'https://www.youtube.com/watch?v=0J2QdDbelmY']

Now we are ready to start the download process by running <code>youtube_dl</code> on our <code>youtube_url_list</code>.

In [4]:
ydl_opts = {
    'format': 'bestaudio/best',
    'nocheckcertificate': False,
    'outtmpl': downloads_folder+youtube_dl.DEFAULT_OUTTMPL,
    'postprocessors': [{'key': 'FFmpegExtractAudio',
                       'preferredcodec': 'mp3',
                       'preferredquality': '192'}],
    'verbose': False,
    }
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    try:
        result = ydl.download(youtube_url_list)
    except Exception as e:
        print(e)
print('Conversion completed.')

[youtube] _nHCM1D9uwk: Downloading webpage
[youtube] _nHCM1D9uwk: Downloading video info webpage
[youtube] _nHCM1D9uwk: Extracting video information
[download] Destination: Downloads\Sam Harris on Russel Brand. Does it matter if we suffer and nobody sees Do we want a God to see-_nHCM1D9uwk.webm
[download] 100% of 15.10MiB in 00:10                  
[ffmpeg] Destination: Downloads\Sam Harris on Russel Brand. Does it matter if we suffer and nobody sees Do we want a God to see-_nHCM1D9uwk.mp3
Deleting original file Downloads\Sam Harris on Russel Brand. Does it matter if we suffer and nobody sees Do we want a God to see-_nHCM1D9uwk.webm (pass -k to keep)
[youtube] PkNv4LFpGf4: Downloading webpage
[youtube] PkNv4LFpGf4: Downloading video info webpage
[youtube] PkNv4LFpGf4: Extracting video information
[download] Destination: Downloads\The Lindsay Shepherd Affair - Update-PkNv4LFpGf4.m4a
[download] 100% of 34.06MiB in 00:17                   
[ffmpeg] Correcting container in "Downloads\The L


Next, we authenticate with Google by running the builtin web server conventiently provided.

In [5]:

from pydrive.auth import GoogleAuth

# The URL that gets displeyed here is not suitable for public consumption
GoogleAuth.DEFAULT_SETTINGS['client_config_file'] = r'../json/client_secrets.json'
gauth = GoogleAuth()
gauth.LocalWebserverAuth()

InvalidConfigError: Invalid client secrets file ('Error opening file', '../json/client_secrets.json', 'No such file or directory', 2)


Before you run the next cell, in the online web page of Google Drive, bring up the share dialog box of the folder you are uploading to and copy its folder id into the <code>tgt_folder_id</code> string. If you right-click on the folder and select **Share...** and then click **Get shareable link**, you will see something like:

https://drive.google.com/drive/folders/1syfUx6jukbW1CWIEy8xoM9veGGr5MBUx?usp=sharing

It's that **1syfUx6jukbW1CWIEy8xoM9veGGr5MBUx** that you need to copy.

In [11]:

from pydrive.drive import GoogleDrive

# Create GoogleDrive instance with authenticated GoogleAuth instance
drive = GoogleDrive(gauth)

# ID of the "Reading Material" folder
tgt_folder_id = '1syfUx6jukbW1CWIEy8xoM9veGGr5MBUh'


Next, we will upload all the mp3s in our download folder.

In [12]:

import os

# Upload all mp3s from the Downloads folder to Google Drive's "Reading Material" folder
gfile_dict = {}
for subdir, dirs, files in os.walk(downloads_folder):
    for src_file in files:
        if src_file.endswith('.mp3'):
            src_path = os.path.join(subdir, src_file)
            gfile_dict[src_file] = drive.CreateFile({'title':src_file, 'mimeType':'audio/mp3',
                                                     'parents': [{'kind': 'drive#fileLink',
                                                                  'id': tgt_folder_id}]})

            # Read mp3 file and set it as a content of this instance.
            gfile_dict[src_file].SetContentFile(src_path)
            
            # Upload the file
            try:
                gfile_dict[src_file].Upload()
                print('Uploaded %s (%s)' % (gfile_dict[src_file]['title'],
                                            gfile_dict[src_file]['mimeType']))
            except Exception as e:
                print('Upload failed for %s (%s): %s' % (gfile_dict[src_file]['title'],
                                                         gfile_dict[src_file]['mimeType'], e))
print('Upload completed.')

Uploaded A Guide to Turning Postmodernism Against Itself-t5mEijjyce4.mp3 (audio/mp3)
Uploaded Political correctness - a force for good A Munk Debate-ST6kj9OEYf0.mp3 (audio/mp3)
Upload completed.



Download the mp3s to <a href='http://www.voicedream.com/support/user-manual/#file'>Voice Dream</a> before you run the trash cell (below).

In [13]:

import os

# Trash all mp3s from Google Drive's "Reading Material" folder
for src_file in gfile_dict.keys():

    # Trash mp3 file
    try:
        gfile_dict[src_file].Trash()
        print('Trashed %s (%s)' % (gfile_dict[src_file]['title'],
                                   gfile_dict[src_file]['mimeType']))
    except Exception as e:
        print('Trash failed for %s (%s): %s' % (gfile_dict[src_file]['title'],
                                                gfile_dict[src_file]['mimeType'], e))
    
print('Trashing completed.')

Trashed A Guide to Turning Postmodernism Against Itself-t5mEijjyce4.mp3 (audio/mp3)
Trashed Political correctness - a force for good A Munk Debate-ST6kj9OEYf0.mp3 (audio/mp3)
Trashing completed.



Lastly, we run the code that will delete all the mp3s out of our downloads folder.

In [14]:

import os

# Delete all mp3s in the Downloads folder
for src_file in gfile_dict.keys():
    src_path = os.path.join(downloads_folder, src_file)

    # Delete the file
    try:
        os.remove(src_path)
        print('Deleted %s (%s)' % (src_file,
                                   gfile_dict[src_file]['mimeType']))
    except Exception as e:
        print('Failed to delete %s (%s): %s' % (src_file,
                                                gfile_dict[src_file]['mimeType'], e))
print('Deleting completed.')

Deleted A Guide to Turning Postmodernism Against Itself-t5mEijjyce4.mp3 (audio/mp3)
Deleted Political correctness - a force for good A Munk Debate-ST6kj9OEYf0.mp3 (audio/mp3)
Deleting completed.
